In [1]:
import numpy as np
import matplotlib.pyplot as plt
from utils import load_imagenet, one_hot
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm, trange
from sklearn.metrics import accuracy_score
import time
import torchvision

GPU = torch.device('cuda:3')
CPU = torch.device('cpu')

In [2]:
(x, y), (x_val, y_val) = load_imagenet()  # 5 parts of 10 parts
m = len(x)
# Show a sample image
# a = x_train.reshape(-1, 3, 64, 64).transpose(0, 2, 3, 1)
# plt.imshow(a[0])

100%|██████████| 10/10 [01:13<00:00,  7.38s/it]


train data loaded
val data loaded
train data converted to tensors


In [3]:
hparams = {
    "batch_size": 256,
    "num_epochs": 40,
    "model": "resnet",
    "dataset": "imagenet",
    "optimizer": "sgd",
    "learning_rate": 0.1,  # 1.2e-5, 1.2e-7 can be tried
}

model = torchvision.models.resnet152()
model.to(GPU)
optimiser = torch.optim.SGD(model.parameters(), lr=hparams["learning_rate"])
losses = []

In [4]:
batch_size = hparams["batch_size"]
num_epochs = hparams["num_epochs"]

ttra, tacc = 0, 0

for epoch in range(num_epochs):

    pbar = tqdm(total=m//batch_size+1, desc=f"Epoch {epoch+1}/{num_epochs}")
    
    t0 = time.time()

    for i in range(0, len(x), batch_size):
        batch_x, batch_y = x[i:i+batch_size].to(GPU), y[i:i+batch_size].to(GPU)
        optimiser.zero_grad()
        output = model(batch_x)
        loss = F.mse_loss(output, batch_y)
        loss.backward()
        optimiser.step()
        pbar.update(1)

    t1 = time.time()

    train_accuracy = accuracy_score(y[:5000].argmax(1),     model(x[:5000].to(GPU)).to(CPU).argmax(1))
    test_accuracy  = accuracy_score(y_val[:5000].argmax(1), model(x_val[:5000].to(GPU)).to(CPU).argmax(1))

    t2 = time.time()

    ttra += t1-t0
    tacc += t2-t1

    losses.append(loss.item())
    torch.save(model, "resnet_imagenet.pth")
    pbar.close()

Epoch 1/40:   1%|▏         | 71/5005 [00:16<18:11,  4.52it/s] 

KeyboardInterrupt: 

In [ ]:
ttra, tacc

In [ ]:
plt.plot(np.arange(len(losses)), losses)

In [ ]:
# torch.save(model, "resnet_imagenet.pth")

In [ ]:
model = torch.load("resnet_imagenet.pth")

In [ ]:
model = model.to(CPU)

In [ ]:
y_pred = model(x[:1000])
y_pred_val = model(x_val[:1000])

train_accuracy = accuracy_score(y[:1000].argmax(1), y_pred.argmax(1))
test_accuracy = accuracy_score(y_val[:1000].argmax(1), y_pred_val.argmax(1))

In [ ]:
print(f"Train accuracy: {train_accuracy*100}")
print(f"Test accuracy: {test_accuracy*100}")